# Evaluation of Trainings on JetNet Dataset on single jettype

## Import, data and checkpoint loading

### Imports

In [ ]:
import os
import sys

sys.path.append("../")

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import hydra
import numpy as np
import pytorch_lightning as pl
import torch
from omegaconf import OmegaConf

In [ ]:
# set env variable DATA_DIR again because of hydra
from dotenv import load_dotenv

load_dotenv()
os.environ["DATA_DIR"] = os.environ.get("DATA_DIR")

In [ ]:
# plots and metrics
import matplotlib.pyplot as plt

from particle_fm.data.components import calculate_all_wasserstein_metrics
from particle_fm.utils.data_generation import generate_data
from particle_fm.utils.plotting import (
    apply_mpl_styles,
    plot_data,
    prepare_data_for_plotting,
)

apply_mpl_styles()

### Load model and datamodule from selected experiment

In [ ]:
experiment = "experiment.yaml"

In [ ]:
# load everything from experiment config
with hydra.initialize(version_base=None, config_path="../configs/"):
    cfg = hydra.compose(config_name="train.yaml", overrides=[f"experiment={experiment}"])
    # print(OmegaConf.to_yaml(cfg))

In [ ]:
datamodule = hydra.utils.instantiate(cfg.data)
model = hydra.utils.instantiate(cfg.model)

In [ ]:
datamodule.setup()

In [ ]:
test_data = np.array(datamodule.tensor_test)
test_mask = np.array(datamodule.mask_test)
test_cond = np.array(datamodule.tensor_conditioning_test)
val_data = np.array(datamodule.tensor_val)
val_mask = np.array(datamodule.mask_val)
val_cond = np.array(datamodule.tensor_conditioning_val)
train_data = np.array(datamodule.tensor_train)
train_mask = np.array(datamodule.mask_train)
train_cond = np.array(datamodule.tensor_conditioning_train)
means = np.array(datamodule.means)
stds = np.array(datamodule.stds)
means_cond = np.array(datamodule.cond_means)
stds_cond = np.array(datamodule.cond_stds)

In [ ]:
print(test_data.shape)
print(test_mask.shape)
print(test_cond.shape)
print(val_data.shape)
print(val_mask.shape)
print(val_cond.shape)
print(train_data.shape)
print(train_mask.shape)
print(train_cond.shape)
print(means)
print(stds)
print(means_cond)
print(stds_cond)

### Load checkpoint

In [ ]:
# add checkpoint path
ckpt = "XXX/checkpoints/last-EMA.ckpt"
model = model.load_from_checkpoint(ckpt)

## Data generation

In [ ]:
dataset = "test"  # choose from "test", "val"
num_samples = (
    -1
)  # negative values are interpreted as multiplications of len(dataset), e.g. -2 -> 2*len(dataset)

In [ ]:
if dataset == "test":
    background_data = test_data
    background_mask = test_mask
    background_cond = test_cond
elif dataset == "val":
    background_data = val_data
    background_mask = val_mask
    background_cond = val_cond
else:
    raise ValueError("Choose from test and val")
print(background_data.shape)
len_data = len(background_data)

In [ ]:
if num_samples < 0:
    factor = abs(num_samples)
    num_samples = len(background_data) * factor
    background_data = np.repeat(background_data, factor, axis=0)
    background_mask = np.repeat(background_mask, factor, axis=0)
    background_cond = np.repeat(background_cond, factor, axis=0)
print(background_data.shape)

In [ ]:
torch.manual_seed(9999)
data, generation_time = generate_data(
    model,
    num_jet_samples=len(background_data),
    batch_size=1000,
    cond=torch.tensor(background_cond),
    variable_set_sizes=True,
    mask=torch.tensor(background_mask),
    normalized_data=True,
    means=means,
    stds=stds,
    ode_solver="em",
    ode_steps=100,
)

In [ ]:
print(f"Generation time: {generation_time:.2f}s")
print(f"Generation time per jet: {generation_time / len(background_data):.5f}s")
print(data.shape)

#### Save data in npy file

In [ ]:
path = "/".join(ckpt.split("/")[:-2]) + "/"
file_name = "generated_data.npy"
full_path = path + file_name
print(full_path)

In [ ]:
np.save(full_path, data)

## Evaluation

### Load data

In [ ]:
path = "/".join(ckpt.split("/")[:-2]) + "/"
# file_name = "generated_data.npy"
file_name = "final_generated_data.npy"
full_path = path + file_name

In [ ]:
data = np.load(full_path)

In [ ]:
print(data.shape)
# data = data[:len_data]
print(data.shape)

### Wasserstein distances

In [ ]:
print(background_data.shape)
print(data.shape)

In [ ]:
w_dists = calculate_all_wasserstein_metrics(
    background_data, data, num_eval_samples=50_000, num_batches=40
)

print(f"W-Dist m: {w_dists['w1m_mean']:4.3E} +- {w_dists['w1m_std']:4.3E}")
print(f"W-Dist p: {w_dists['w1p_mean']:4.3E} +- {w_dists['w1p_std']:4.3E}")
print(f"W-Dist efp: {w_dists['w1efp_mean']:4.3E} +- {w_dists['w1efp_std']:4.3E}")

### Plots

In [ ]:
plot_config = {
    "num_samples": -1,
    "plot_jet_features": True,
    "plot_w_dists": False,
    "plot_efps": False,
    "plot_selected_multiplicities": True,
    "selected_multiplicities": [10, 20, 30, 40, 50, 100],
    "selected_particles": [1, 3, 10],
    "plottype": "sim_data",
    "save_fig": False,
    "variable_jet_sizes_plotting": True,
    "bins": 100,
    "close_fig": False,
}
plot_prep_config = {
    "calculate_efps" if key == "plot_efps" else key: value
    for key, value in plot_config.items()
    if key in ["plot_efps", "selected_particles", "selected_multiplicities"]
}

In [ ]:
(
    jet_data,
    efps_values,
    pt_selected_particles,
    pt_selected_multiplicities,
) = prepare_data_for_plotting(
    np.array([data]),
    **plot_prep_config,
)

In [ ]:
(
    jet_data_sim,
    efps_sim,
    pt_selected_particles_sim,
    pt_selected_multiplicities_sim,
) = prepare_data_for_plotting(
    [background_data],
    **plot_prep_config,
)
jet_data_sim, efps_sim, pt_selected_particles_sim = (
    jet_data_sim[0],
    efps_sim[0],
    pt_selected_particles_sim[0],
)

In [ ]:
fig = plot_data(
    particle_data=np.array([data]),
    sim_data=background_data,
    jet_data_sim=jet_data_sim,
    jet_data=jet_data,
    efps_sim=efps_sim,
    efps_values=efps_values,
    pt_selected_particles=pt_selected_particles,
    pt_selected_multiplicities=pt_selected_multiplicities,
    pt_selected_particles_sim=pt_selected_particles_sim,
    pt_selected_multiplicities_sim=pt_selected_multiplicities_sim,
    **plot_config,
)

### Truth value wasserstein distance

In [ ]:
w_dists = calculate_all_wasserstein_metrics(
    background_data, train_data, num_eval_samples=50_000, num_batches=40
)

print(f"W-Dist m: {w_dists['w1m_mean']:4.3E} +- {w_dists['w1m_std']:4.3E}")
print(f"W-Dist p: {w_dists['w1p_mean']:4.3E} +- {w_dists['w1p_std']:4.3E}")
print(f"W-Dist efp: {w_dists['w1efp_mean']:4.3E} +- {w_dists['w1efp_std']:4.3E}")

### Jet substructure

#### create and save substructure data

In [ ]:
import h5py

from particle_fm.data.components.metrics import wasserstein_distance_batched
from particle_fm.utils.jet_substructure import dump_hlvs

In [ ]:
path = "/".join(ckpt.split("/")[:-2]) + "/"
file_name = "substructure"
full_path = path + file_name
print(full_path)

In [ ]:
dump_hlvs(data, full_path, plot=True)

#### JetNet data

In [ ]:
# define path for jetnet substructure
path_jetnet = f"XXX/jetnet_substructure/{datamodule.hparams.jet_type[0]}-{datamodule.hparams.num_particles}_substructure"

In [ ]:
dump_hlvs(background_data, path_jetnet, plot=True)

#### load and compare generated data with JetNet data (w-dists)

In [ ]:
# load substructure for model generated data
keys = []
data_substructure = []
with h5py.File(full_path + ".h5", "r") as f:
    print(f.keys())
    tau21 = np.array(f["tau21"])
    tau32 = np.array(f["tau32"])
    d2 = np.array(f["d2"])
    for key in f.keys():
        keys.append(key)
        data_substructure.append(np.array(f[key]))
data_substructure = np.array(data_substructure)

In [ ]:
# load substructure for JetNet data
data_substructure_jetnet = []
with h5py.File(path_jetnet + ".h5", "r") as f:
    tau21_jetnet = np.array(f["tau21"])
    tau32_jetnet = np.array(f["tau32"])
    d2_jetnet = np.array(f["d2"])
    for key in f.keys():
        data_substructure_jetnet.append(np.array(f[key]))
data_substructure_jetnet = np.array(data_substructure_jetnet)

In [ ]:
w_dist_tau21_mean, w_dist_tau21_std = wasserstein_distance_batched(
    tau21_jetnet, tau21, num_eval_samples=50_000, num_batches=40
)
w_dist_tau32_mean, w_dist_tau32_std = wasserstein_distance_batched(
    tau32_jetnet, tau32, num_eval_samples=50_000, num_batches=40
)
w_dist_d2_mean, w_dist_d2_std = wasserstein_distance_batched(
    d2_jetnet, d2, num_eval_samples=50_000, num_batches=40
)
print(f"W-Dist tau21: {w_dist_tau21_mean:4.3E} +- {w_dist_tau21_std:4.3E}")
print(f"W-Dist tau32: {w_dist_tau32_mean:4.3E} +- {w_dist_tau32_std:4.3E}")
print(f"W-Dist d2: {w_dist_d2_mean:4.3E} +- {w_dist_d2_std:4.3E}")

## Truth data

In [ ]:
print(train_data.shape)
print(test_data.shape)

In [ ]:
path_train = "XXX"
dump_hlvs(train_data, path_train, plot=True)

In [ ]:
path_test = "XXX"
dump_hlvs(test_data, path_test, plot=True)

In [ ]:
# load substructure for model generated data
keys = []
data_substructure = []
with h5py.File(path_train + ".h5", "r") as f:
    print(f.keys())
    tau21 = np.array(f["tau21"])
    tau32 = np.array(f["tau32"])
    d2 = np.array(f["d2"])
    for key in f.keys():
        keys.append(key)
        data_substructure.append(np.array(f[key]))
data_substructure = np.array(data_substructure)

In [ ]:
# load substructure for JetNet data
data_substructure_jetnet = []
with h5py.File(path_test + ".h5", "r") as f:
    tau21_jetnet = np.array(f["tau21"])
    tau32_jetnet = np.array(f["tau32"])
    d2_jetnet = np.array(f["d2"])
    for key in f.keys():
        data_substructure_jetnet.append(np.array(f[key]))
data_substructure_jetnet = np.array(data_substructure_jetnet)

In [ ]:
print(tau21_jetnet.shape)
print(tau21.shape)
print(test_data.shape)

In [ ]:
w_dist_tau21_mean, w_dist_tau21_std = wasserstein_distance_batched(
    tau21_jetnet, tau21, num_eval_samples=50_000, num_batches=40
)
w_dist_tau32_mean, w_dist_tau32_std = wasserstein_distance_batched(
    tau32_jetnet, tau32, num_eval_samples=50_000, num_batches=40
)
w_dist_d2_mean, w_dist_d2_std = wasserstein_distance_batched(
    d2_jetnet, d2, num_eval_samples=50_000, num_batches=40
)
print(f"W-Dist tau21: {w_dist_tau21_mean:4.3E} +- {w_dist_tau21_std:4.3E}")
print(f"W-Dist tau32: {w_dist_tau32_mean:4.3E} +- {w_dist_tau32_std:4.3E}")
print(f"W-Dist d2: {w_dist_d2_mean:4.3E} +- {w_dist_d2_std:4.3E}")